In [1]:
import numpy as np
import os
import tqdm

# Reading features that were extracted previously

In [2]:
X = []
Y = []

for file in tqdm.tqdm(os.listdir('npy-features/')):
    for x in np.load('npy-features/' + file, allow_pickle=True):
        X.append(x[0])
        Y.append(x[1])

100%|██████████| 7/7 [00:00<00:00, 13.13it/s]


In [3]:
n = len(X)
print(n)

66806


In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2)

In [7]:
n_train = len(x_train)
n_test = len(x_test)

In [8]:
print(n_train)
print(n_test)

53444
13362


In [9]:
labels = ['Bank account or service',
 'Consumer Loan',
 'Credit card',
 'Credit reporting',
 'Debt collection',
 'Money transfers',
 'Mortgage',
 'Other financial service',
 'Payday loan',
 'Prepaid card',
 'Student loan']

In [10]:
def label_maker(x):
    for i,c in enumerate(labels):
        if c==x:
            return i

In [11]:
n_labels = len(labels)
n_labels

11

In [12]:
len(x_train)

53444

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Dense, Flatten,Conv1D, MaxPooling1D

In [14]:
y_train = np.asarray(y_train, dtype='int8')
y_test = np.asarray(y_test, dtype='int8')

In [15]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)
print(x_train.shape)
x_test[0]

(53444, 50)


array([ 1.88708758e+01,  2.85622012e+00, -2.42515947e+01,  1.14139290e+01,
       -1.33398030e+01,  2.30068843e+00, -2.81365419e+00, -8.64853982e+00,
       -6.64017134e-01, -1.18037336e-01, -3.07575108e+00,  7.64792212e+00,
       -1.61940202e+00, -6.68069888e+00,  7.95706108e+00, -1.68956075e+00,
       -3.66671441e+00,  2.64936986e+00,  1.46747150e-01, -6.47017166e+00,
       -6.17110068e+00,  1.07104239e-01,  7.02590284e+00,  1.91660488e+00,
        2.15427757e-01,  4.43324316e+00, -9.10517277e+00, -4.14407989e+00,
        4.22121095e+00, -3.17312605e+00, -2.68538301e+00,  9.67576097e+00,
        1.88282142e+00, -3.90372677e+00,  1.55911828e+00, -6.18145522e-03,
        5.16647439e+00, -6.68452209e+00, -7.16660914e+00,  2.88125779e+00,
       -2.07801562e+00, -1.91162237e+00, -4.13302508e+00, -1.14468062e+01,
       -3.38594042e+00, -1.29510322e+00,  1.26183948e+01, -3.28793121e+00,
       -8.23622282e-01, -1.08654328e+00])

In [55]:
x_train = x_train.reshape((len(x_train),50))
x_test = x_test.reshape((len(x_test),50))

In [56]:
import tensorflow as tf
tf.test.is_gpu_available()

False

# Feed forward Neural Network Model

In [66]:
model = Sequential()

model.add(Dense(50, input_dim=50, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(labels), activation='softmax'))

In [67]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.fit(x_train, y_train, batch_size=128, epochs=10)

Epoch 1/10
 7680/53444 [===>..........................] - ETA: 0s - loss: 0.8175 - acc: 0.7440

In [69]:
model.evaluate(x_test, y_test, batch_size=64)

13362/13362 [==============================] - 0s 19us/sample - loss: 0.7674 - acc: 0.7536


[0.7673725894407105, 0.7536297]

In [20]:
predictions = model.predict_classes(x_test)

In [21]:
import nltk
from nltk.corpus import stopwords
stopword = set(stopwords.words('english'))
from nltk import word_tokenize
import re
from itertools import chain
from gensim.models import Word2Vec
from joblib import load, dump

In [22]:
transformer = load('PCA.joblib')
print(transformer)

IncrementalPCA(batch_size=50, copy=True, n_components=50, whiten=False)


In [23]:
embedding = Word2Vec.load('w2v_embedding.model')
print(embedding)

Word2Vec(vocab=11097, size=50, alpha=0.025)


/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [24]:
def preprocess(x):
    x = x.lower()
    x = re.sub(r'[^\w]', ' ', x)
    x = word_tokenize(x)
    x = [w for w in x if w not in stopword]
    if len(x) > 60:
        x = x[:60]
    for j in range(60-len(x)):
            x.append(' ')

    for i, word in enumerate(x):
        if word in embedding.wv.vocab.keys():
            x[i] = embedding[word]
        else:
            x[i] = [0]*50    
    x = list(chain.from_iterable(x))
    x = np.asarray(x).reshape(1, len(x))
    x = transformer.transform(x)
    
    return x

In [25]:
x = ''
print(x)

In [26]:
def predict(x):
    x = preprocess(x)
    return labels[model.predict_classes(x)[0]]

In [27]:
predict('')

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


'Credit reporting'

In [28]:
model.evaluate(x_test, y_test)

13362/13362 [==============================] - 0s 17us/sample - loss: 0.7728 - acc: 0.7491


[0.7727767728372813, 0.74913937]

In [29]:
def save_model(model_file,weights):
	json = model.to_json()
	with open(model_file, "w") as json_file:
		json_file.write(json)
	model.save_weights(weights)
	print(model, 'saved.')

In [50]:
model_ = load_model('model-v1b1.json', 'model-v1b1.h5')

<tensorflow.python.keras.engine.sequential.Sequential object at 0x7f6dc2681590> has been loaded.


In [53]:
model_.predict_classes(preprocess('hi'))

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


array([3])

In [30]:
def load_model(json_file,weights): #load x model json, y
    #LOAD MODEL
    from tensorflow.keras.models import model_from_json

    json = open(json_file,'r')
    model = json.read()
    model = model_from_json(model)
    model.load_weights(weights)
    print(model,'has been loaded.')
    return model


In [45]:
save_model('model-v1b1.json', 'model-v1b1.h5')

<tensorflow.python.keras.engine.sequential.Sequential object at 0x7f6e4a904710> saved.


In [54]:
model.save('model.h5')

In [31]:
df = pd.read_csv('consumer_complaints.csv')

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
import re
from nltk import word_tokenize

In [82]:
def predict(x):
    i = model.predict_classes(preprocess(x))[0]
    print(i)
    return labels[i]

In [80]:
pd.read_csv('consumer_complaints.csv')

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
555952,07/01/2014,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,Bank of America,NaN,NaN,NaN,NaN,Referral,07/07/2014,Closed with explanation,Yes,No,919529
555953,07/01/2014,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Residential Credit Solutions,NaN,NaN,"Older American, Servicemember",NaN,Referral,07/23/2014,Closed with explanation,No,No,918447
555954,07/10/2012,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,BB&T Financial,NaN,NaN,NaN,NaN,Phone,11/18/2013,Closed with explanation,Yes,No,114550
555955,04/14/2015,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,NaN,NaN,Global Recovery Group,NaN,NaN,NaN,NaN,Phone,04/14/2015,Untimely response,No,No,1329963


In [1]:
import pandas as pd
df = pd.read_csv('consumer_complaints.csv')

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df = df.fillna('na')

In [8]:
x = df[df['consumer_complaint_narrative'] != 'na']

In [14]:
x[x['product'] == 'Student loan'].tail(50)

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
545397,01/12/2016,Student loan,Non-federal student loan,Can't repay my loan,Can't decrease my monthly payments,I have contacted Sallie Mae and my mother abou...,Company believes it acted appropriately as aut...,Sallie Mae,AL,350XX,na,Consent provided,Web,01/12/2016,Closed with explanation,Yes,No,1738638
545403,12/25/2015,Student loan,Non-federal student loan,Dealing with my lender or servicer,Trouble with how payments are handled,I have XXXX loan which has the same account nu...,na,Discover,AZ,852XX,na,Consent provided,Web,12/25/2015,Closed with explanation,Yes,No,1716589
545453,01/19/2016,Student loan,Non-federal student loan,Dealing with my lender or servicer,Keep getting calls about my loan,"Private student loan with Sallie Mae, the amou...",Company believes it acted appropriately as aut...,Sallie Mae,AL,350XX,na,Consent provided,Web,01/19/2016,Closed with explanation,Yes,No,1747517
545550,04/01/2016,Student loan,Non-federal student loan,Getting a loan,Can't qualify for a loan,My credit report is showing me as having the s...,Company has responded to the consumer and the ...,Experian,FL,330XX,na,Consent provided,Web,04/02/2016,Closed with non-monetary relief,Yes,No,1861059
545878,01/09/2016,Student loan,Non-federal student loan,Dealing with my lender or servicer,Having problems with customer service,"Hello, My student loans were originally servic...",na,Discover,PA,190XX,na,Consent provided,Web,01/13/2016,Closed with explanation,Yes,No,1735076
546070,02/05/2016,Student loan,Non-federal student loan,Dealing with my lender or servicer,Don't agree with fees charged,the company Campus Partners services private s...,na,Heartland Payment Systems,OH,432XX,na,Consent provided,Web,02/05/2016,Closed with explanation,Yes,No,1775839
546125,01/07/2016,Student loan,Non-federal student loan,Dealing with my lender or servicer,Having problems with customer service,I am a navient student loan customer and curre...,na,"Navient Solutions, Inc.",TX,752XX,na,Consent provided,Web,01/07/2016,Closed with explanation,Yes,No,1732629
546299,02/05/2016,Student loan,Non-federal student loan,Dealing with my lender or servicer,Keep getting calls about my loan,Have been harassed and bombarded with numerous...,Company chooses not to provide a public response,Sunrise Credit Services Inc.,MA,015XX,na,Consent provided,Web,02/09/2016,Closed with explanation,Yes,No,1774338
546514,12/12/2015,Student loan,Non-federal student loan,Dealing with my lender or servicer,Having problems with customer service,I submitted a request to American Education Se...,na,AES/PHEAA,PA,172XX,na,Consent provided,Web,12/12/2015,Closed with explanation,Yes,No,1694817
546757,01/31/2016,Student loan,Non-federal student loan,Can't repay my loan,Can't temporarily postpone payments,I called Sallie Mae to request forbearance whi...,Company believes complaint is the result of an...,Sallie Mae,MO,656XX,na,Consent provided,Web,01/31/2016,Closed with non-monetary relief,Yes,No,1765952


In [15]:

x[x['complaint_id'] == 1716589]['consumer_complaint_narrative'].item()

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `item` has been deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


'I have XXXX loan which has the same account number with Great Lakes, XXXX XXXX XXXX, XXXX, WI XXXX. I have been paying almost every 2 weeks with {$150.00} and they were crediting them to the lowest interest ( 7 % ) and showing overdue on the highest interest rate ( 9 % ). I called them several times to resolve by consolidating or lowering my interest rate ; they have rejected my request neither they are willing to talk. My account number is XXXX. I would request the esteemed Attorney General or their office to intervene to resolve this issue as I am struggling with all the highest payments. \n'

In [3]:
print(df[df['complaint_id'] == 1869939]['consumer_complaint_narrative'].item())
df[df['complaint_id'] == 1869939]['product'].item()


I was mislead on all information on the predatory school loans provided by Genesis Lending XXXX. XXXX XXXX XXXX XXXX. XXXX XXXX. 



/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `item` has been deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: `item` has been deprecated and will be removed in a future version
  


'Student loan'

In [42]:
df[df['consumer_complaint_narrative'] != 'na'].tail(50)

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
552848,02/08/2016,Mortgage,FHA mortgage,"Loan modification,collection,foreclosure",na,To : OCC From : XXXX XXXX XXXX RE : Short Sale...,Company chooses not to provide a public response,Citibank,GA,301XX,na,Consent provided,Web,02/08/2016,Closed with explanation,Yes,No,1778911
552854,02/17/2016,Bank account or service,Checking account,"Making/receiving payments, sending money",na,"On XXXX XXXX, 2016, I initiated a "" Send Money...",Company believes it acted appropriately as aut...,USAA Savings,NY,100XX,Servicemember,Consent provided,Web,02/17/2016,Closed with monetary relief,Yes,No,1790591
552865,02/01/2016,Bank account or service,Other bank product/service,"Making/receiving payments, sending money",na,I paid via Paypal a friend for a game currency...,na,"PayPal Holdings, Inc.",IN,464XX,na,Consent provided,Web,02/01/2016,Closed with explanation,Yes,No,1766786
552871,04/07/2016,Student loan,Non-federal student loan,Can't repay my loan,Can't get flexible payment options,I was mislead on all information on the predat...,Company believes complaint caused principally ...,Genesis Lending,ND,588XX,na,Consent provided,Web,04/07/2016,Closed with explanation,Yes,No,1869939
552874,02/08/2016,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt was paid,I was liviing in XXXX NJ and subsequent to bec...,na,"Convergent Resources, Inc.",NJ,078XX,na,Consent provided,Web,02/10/2016,Closed with explanation,Yes,No,1776730
552875,02/09/2016,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",na,Please see attached letter to Wells Fargo Home...,Company chooses not to provide a public response,Wells Fargo & Company,CA,921XX,na,Consent provided,Web,02/09/2016,Closed with explanation,Yes,No,1779322
552876,01/25/2016,Debt collection,"Other (i.e. phone, health club, etc.)",False statements or representation,Attempted to collect wrong amount,Central Credit Services is reporting the same ...,Company chooses not to provide a public response,Radius Global Solutions LLC,AZ,852XX,na,Consent provided,Web,01/28/2016,Closed with explanation,Yes,No,1756174
552882,03/31/2016,Debt collection,Credit card,Taking/threatening an illegal action,Seized/Attempted to seize property,I was 6 months late with XXXX Wells Fargo cred...,Company has responded to the consumer and the ...,Wells Fargo & Company,CA,923XX,na,Consent provided,Web,04/04/2016,Closed with explanation,Yes,No,1858208
552884,02/10/2016,Debt collection,Auto,Communication tactics,Threatened to take legal action,I just received a phone call from a woman who ...,na,Santander Consumer USA Holdings Inc,AE,090XX,Servicemember,Consent provided,Web,02/10/2016,Closed with explanation,Yes,No,1781287
552887,02/06/2016,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",na,Ditech collects escrow monthly ( {$240.00} ) a...,Company believes complaint is the result of an...,Ditech Financial LLC,NC,276XX,na,Consent provided,Web,02/06/2016,Closed with non-monetary relief,Yes,No,1776308


In [80]:
predict('incorrect pin in atm')

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


'Credit reporting'

In [81]:
predict('Central Credit Services is reporting the same debt multiple times on my credit report. I dispute the validity of the debt as they also never had the correct address or phone number until I recently contacted them after viewing my credit report. But without question, they have reported the same debt 3 times on my XXXX Credit Report.')

/home/yatharthmathur/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


'Debt collection'